# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [19]:
# import numpy and pandas
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from scipy.stats import f_oneway

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [1]:
# Your code here:
#pokemon.head()


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:
pokemon["legendary_count"] = pokemon.groupby("Legendary").cumcount()
#pokemon["legendary_count"]

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
# Your code here:


ds_pivot = pokemon.pivot(index='legendary_count', columns='Legendary', values='Total')
ds_pivot.columns = ['Legendary_'+str(x) for x in ds_pivot.columns.values]
#ds_pivot


# legendary: mean = 637.4 and std = 60.9
# non_legendary: mean = 417.2 and std = 106.8

The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [6]:
# Your code here:

ttest_ind(ds_pivot["Legendary_False"], ds_pivot["Legendary_True"], equal_var=False, nan_policy='omit')

Ttest_indResult(statistic=-25.8335743895517, pvalue=9.357954335957464e-47)

What do you conclude from this test? Write your conclusions below.

In [7]:
# Your conclusions here:
#I can conclude that by the p-value there are significant differences between the lengendary and non Legendary pokemons.


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [2]:
# Your code here:
count = pokemon.groupby("Type 1", as_index=True).agg({"Type 1" : "count"})
#count


NameError: name 'pokemon' is not defined

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [3]:
# Your code here:

"""
What i need to do:
1) create a variable with the water values
2) create a variable that is the agg of the outher values
3) preform the analisys
"""

pokemon["type_count"] = pokemon.groupby("Type 1").cumcount()
ds_pivot1 = pokemon.pivot(index='type_count', columns='Type 1', values='Total')
ds_pivot1.columns = [str(x) for x in ds_pivot1.columns.values]

new = ds_pivot1.loc[:, ds_pivot1.columns != 'Water']
#new

NameError: name 'pokemon' is not defined

Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [10]:
# Your code here:

ttest_ind(ds_pivot1["Water"], ds_pivot1, equal_var=True, nan_policy='omit')[1]



masked_array(data=[3.85487817e-03, 5.03622889e-01, 2.13521665e-06,
                   5.13443326e-01, 5.63364340e-01, 5.57825959e-01,
                   1.44019156e-01, 3.52070445e-01, 6.86962903e-01,
                   5.81639254e-01, 7.61585793e-01, 9.05597831e-01,
                   7.04218826e-02, 1.77845234e-01, 2.36809706e-02,
                   2.43289924e-01, 2.01893049e-02, 1.00000000e+00],
             mask=False,
       fill_value=1e+20)

Write your conclusion below.

In [11]:
# Your conclusions here:

## Looking at the p-values we can see that there are significative differences between the groups when compared with the water.

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [24]:
# Your code here:
#H0 is atack==defese "

from scipy.stats import ttest_rel
ttest_rel(pokemon["Defense"], pokemon["Attack"])

Describe the results of the test in the cell below.

In [12]:
# Your conclusions here:
## The p value is very small therefore the hypotesis is rejected.


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [20]:
# Your code here:

ttest_rel(pokemon["Sp. Def"], pokemon["Sp. Atk"])

Ttest_relResult(statistic=-0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [14]:
# Your conclusions here:
# The p-value is > than 0.05 so there are no significant differences.


As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [27]:
# Your code here:
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
ttest_1samp(pokemon['Defense'] - pokemon['Attack'], 0)  == ttest_rel(pokemon["Defense"], pokemon["Attack"])



True

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [17]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [18]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:

